The circuit topology:

<img src="3954.png" alt="Circuit 3954" width="200" />

The circuit PDEs:

$
     \frac{\partial A}{\partial t} = b_A  +   V_A  \frac{1}{1+\left(\frac{K_{AA}}{A }\right)^{n}} \frac{1}{1+\left(\frac{B}{K_{BA} }\right)^{n}} -  \mu_A A + D_A \nabla^2 A,
$

$     \frac{\partial B}{\partial t} = b_B  +   V_B  \frac{1}{1+\left(\frac{K_{AB}}{A }\right)^{n}} \frac{1}{1+\left(\frac{C}{K_{CB} }\right)^{n}} -  \mu_B B + D_B \nabla^2 B,
$

$
\frac{\partial C}{\partial t} = b_C  +   V_C  \frac{1}{1+\left(\frac{A}{K_{AC} }\right)^{n}} \frac{1}{1+\left(\frac{B}{K_{BC} }\right)^{n}} \frac{1}{1+\left(\frac{K_{CC}}{C }\right)^{n}} -  \mu_C C,
$

Where, kinetic terms are

$f_A(A, B, C) = b_A  +   V_A  \frac{1}{1+\left(\frac{K_{AA}}{A }\right)^{n}} \frac{1}{1+\left(\frac{B}{K_{BA} }\right)^{n}} -  \mu_A A$

$f_B(A, B, C) = b_B  +   V_B  \frac{1}{1+\left(\frac{K_{AB}}{A }\right)^{n}} \frac{1}{1+\left(\frac{C}{K_{CB} }\right)^{n}} -  \mu_B B$

$f_C(A, B, C) = b_C  +   V_C  \frac{1}{1+\left(\frac{A}{K_{AC} }\right)^{n}} \frac{1}{1+\left(\frac{B}{K_{BC} }\right)^{n}} \frac{1}{1+\left(\frac{K_{CC}}{C }\right)^{n}} -  \mu_C C$

First, load the parameters

In [1]:
import pandas as pd

df_parameters = pd.read_csv("./Turing_parameters.csv")
# Showing the top 5 rows
df_parameters.head()

,D_A,D_B,D_C,n,b_A,mu_A,V_A,K_AA,K_AB,K_AC,...,K_BA,K_BC,b_C,mu_C,V_C,K_CB,K_CC,A,B,C
0,0.01,1.0,0.0,4.0,0.001,0.1,5.0,0.5,0.5,0.5,...,10.0,0.5,0.1,0.5,0.5,1.0,0.5,0.518949,26.823553,0.200000
1,0.01,1.0,0.0,4.0,0.001,0.1,5.0,0.5,1.0,0.5,...,0.5,0.5,20.0,100.0,100.0,10.0,0.5,0.403373,1.231664,0.200468
2,0.01,1.0,0.0,4.0,0.001,0.1,5.0,0.5,1.0,1.0,...,0.5,0.5,0.1,0.5,20.0,20.0,1.0,0.377202,1.192112,0.201954
3,0.01,1.0,0.0,4.0,0.001,0.1,5.0,0.5,1.0,1.0,...,1.0,0.5,0.1,0.5,10.0,5.0,1.0,0.388113,2.418637,0.200057
4,0.01,1.0,0.0,4.0,0.001,0.1,5.0,0.5,0.5,1.0,...,0.5,0.5,1.0,5.0,100.0,50.0,1.0,0.160812,0.629342,0.211336


The parameters on, say row``=index``, can be parsed as follows

In [2]:
index = 0
(D_A, D_B, D_C, 
 n,
 b_A, mu_A, V_A, K_AA, K_AB, K_AC,
 b_B, mu_B, V_B, K_BA, K_BC,
 b_C, mu_C, V_C, K_CB, K_CC,
 A_star, B_star, C_star) = df_parameters.iloc[index]

And finally, you can create the **Kinetic** function by providing the dataframe and the row index using the following code. Note that the ``create_kinetics`` is a higher-order function in Python, which returns a function.

In [3]:
def act(x, K, n):
    """Activation"""
    return x**n / (x**n + K**n)


def inh(x, K, n):
    """Inhibition"""
    return K**n / (K**n + x** n)

def create_kinetics(df, index):
    (_, _, _, 
    n,
    b_A, mu_A, V_A, K_AA, K_AB, K_AC,
    b_B, mu_B, V_B, K_BA, K_BC,
    b_C, mu_C, V_C, K_CB, K_CC,
    _, _, _) = df.iloc[index]
    def kinetics(A, B, C):
        fA_v = b_A + V_A * act(A, K_AA, n) * inh(B, K_BA, n) - mu_A * A
        fB_v = b_B + V_B * act(A, K_AB, n) * inh(C, K_CB, n) - mu_B * B
        fC_v = b_C + V_C * inh(A, K_AC, n) * inh(B, K_BC, n) * act(C, K_CC, n) - mu_C * C
        return (fA_v, fB_v, fC_v)
    return kinetics


For example, to create a kinetic function for the parameter in the first row, we do the following:

In [4]:
kinetic =  create_kinetics(df_parameters, index=0)

To check the steady state solutions, see the followings:

In [5]:
A_star, B_star, C_star = df_parameters.iloc[0]["A"], df_parameters.iloc[0]["B"], df_parameters.iloc[0]["C"]
print(f"A:{A_star:.4f}, B:{B_star:.4f}, C:{A_star:.4f}")
print(f"kinetics: {kinetic(A_star, B_star, C_star)}")

A:0.5189, B:26.8236, C:0.5189
kinetics: (-1.3436546320022558e-10, -2.209541438702445e-09, 0.0)
